In [15]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column
import pandas as pd

# 1️⃣ Leer datasets
asthma = pd.read_csv("NYC EH Data Portal - Adults with asthma (past 12 months) (full table).csv")
pm25 = pd.read_csv("NYC EH Data Portal - Fine particles (PM 2.5) (full table).csv")

# 2️⃣ Clean columns (remove '*' and convert to numeric)
asthma['Percent_clean'] = asthma['Percent'].str.extract(r'(\d+\.\d+)').astype(float)
asthma['TimePeriod'] = asthma['TimePeriod'].astype(int)

pm25['Annual_mean'] = pm25['Annual mean mcg/m3']
pm25['TimePeriod'] = pm25['TimePeriod'].astype(int)

# 3️⃣ Group both by year (mean)
asthma_yearly = asthma.groupby('TimePeriod')['Percent_clean'].mean().reset_index()
pm25_yearly = pm25.groupby('TimePeriod')['Annual_mean'].mean().reset_index()

# 4️⃣ Normalize both series (Min-Max scaling)
asthma_yearly['Percent_norm'] = (asthma_yearly['Percent_clean'] - asthma_yearly['Percent_clean'].min()) / (asthma_yearly['Percent_clean'].max() - asthma_yearly['Percent_clean'].min())
pm25_yearly['Annual_norm'] = (pm25_yearly['Annual_mean'] - pm25_yearly['Annual_mean'].min()) / (pm25_yearly['Annual_mean'].max() - pm25_yearly['Annual_mean'].min())

# 5️⃣ Prepare ColumnDataSources
source_asthma = ColumnDataSource(asthma_yearly)  # Fuente original de Asma
source_pm25 = ColumnDataSource(pm25_yearly)

# 6️⃣ Fuente de datos desplazada para la línea de Asma (esto se moverá con el slider)
source_asthma_shifted = ColumnDataSource(data=dict(TimePeriod=asthma_yearly['TimePeriod'], Percent_norm=asthma_yearly['Percent_norm']))

# 7️⃣ Crear figura
p = figure(title='Normalized Asma vs PM2.5 with Temporal Shift',
           x_axis_label='Year',
           y_axis_label='Normalized Value (0-1)',
           width=900, height=500,
           tools='pan,wheel_zoom,box_zoom,reset,save')

# Asma → naranja (#ff7f0e) (Línea desplazable)
line1 = p.line('TimePeriod', 'Percent_norm', source=source_asthma_shifted, color='#ff7f0e', legend_label='Asma (Normalized)', line_width=3)
circle1 = p.circle('TimePeriod', 'Percent_norm', source=source_asthma_shifted, color='#ff7f0e', size=8)

# PM2.5 → morado (#9467bd)
line2 = p.line('TimePeriod', 'Annual_norm', source=source_pm25, color='#9467bd', legend_label='PM2.5 (Normalized)', line_width=3)
circle2 = p.circle('TimePeriod', 'Annual_norm', source=source_pm25, color='#9467bd', size=8)

# 8️⃣ Hover Tool para Asma (usar datos originales de Asma)
hover_asthma = HoverTool()
hover_asthma.tooltips = [
    ('Year', '@TimePeriod'),
    ('Asma Value', '@Percent_norm')
]
# Asociar el hover solo a la línea y círculos de Asma desplazada
hover_asthma.renderers = [line1, circle1]
p.add_tools(hover_asthma)

# 9️⃣ Hover Tool para PM2.5 (usar datos originales de PM2.5)
hover_pm25 = HoverTool()
hover_pm25.tooltips = [
    ('Year', '@TimePeriod'),
    ('PM2.5 Value', '@Annual_norm')
]
# Asociar el hover solo a la línea y círculos de PM2.5
hover_pm25.renderers = [line2, circle2]
p.add_tools(hover_pm25)

# 10️⃣ Interactividad en el gráfico (desaparecer elementos al hacer clic)
p.legend.click_policy = "hide"

# 🔥 Agregar Slider para desplazar la línea de Asma
shift_slider = Slider(start=-5, end=5, value=0, step=1, title="Shift Asma Curve (Years)")

# Definir la función de desplazamiento con CustomJS (no modificar la fuente original)
callback = CustomJS(args=dict(source=source_asthma_shifted, slider=shift_slider,
                               original_x=asthma_yearly['TimePeriod'].tolist(),
                               original_y=asthma_yearly['Percent_norm'].tolist()), code=""" 
    const data = source.data;
    const shift = slider.value;
    const x = original_x;
    const y = original_y;
    const new_x = [];
    for (let i = 0; i < x.length; i++) {
        new_x.push(x[i] + shift);
    }
    data['TimePeriod'] = new_x;
    source.change.emit();
""")

shift_slider.js_on_change('value', callback)

# 🔥 Mostrar gráfico con el slider
show(column(p, shift_slider))


In [29]:
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import HoverTool, ColumnDataSource, Slider, CustomJS, LabelSet, CheckboxButtonGroup
from bokeh.layouts import column
import pandas as pd

# 1️⃣ Leer datasets
asthma = pd.read_csv("NYC EH Data Portal - Adults with asthma (past 12 months) (full table).csv")
pm25 = pd.read_csv("NYC EH Data Portal - Fine particles (PM 2.5) (full table).csv")

# 2️⃣ Limpiar columnas
asthma['Percent_clean'] = asthma['Percent'].str.extract(r'(\d+\.\d+)').astype(float)
asthma['TimePeriod'] = asthma['TimePeriod'].astype(int)

pm25['Annual_mean'] = pm25['Annual mean mcg/m3']
pm25['TimePeriod'] = pm25['TimePeriod'].astype(int)

# 3️⃣ Encontrar años comunes
common_years = set(asthma['TimePeriod']) & set(pm25['TimePeriod'])

# 4️⃣ Filtrar los datasets para solo los años comunes
asthma_yearly_filtered = asthma[asthma['TimePeriod'].isin(common_years)].reset_index(drop=True)
pm25_yearly_filtered = pm25[pm25['TimePeriod'].isin(common_years)].reset_index(drop=True)

# 5️⃣ Agrupar por año (media)
asthma_yearly = asthma_yearly_filtered.groupby('TimePeriod')['Percent_clean'].mean().reset_index()
pm25_yearly = pm25_yearly_filtered.groupby('TimePeriod')['Annual_mean'].mean().reset_index()

# 6️⃣ Normalizar Asma
asthma_yearly['Percent_norm'] = (asthma_yearly['Percent_clean'] - asthma_yearly['Percent_clean'].min()) / (asthma_yearly['Percent_clean'].max() - asthma_yearly['Percent_clean'].min())

# 7️⃣ Calcular cigarrillos totales al año para PM2.5
pm25_yearly['Cigarettes_per_day'] = pm25_yearly['Annual_mean'] / 22
pm25_yearly['Cigarettes_per_year'] = (pm25_yearly['Cigarettes_per_day'] * 365).round(0).astype(int)

# 8️⃣ Preparar ColumnDataSources
source_asthma = ColumnDataSource(asthma_yearly)
source_pm25 = ColumnDataSource(pm25_yearly)

# 9️⃣ Fuente desplazada para Asma
source_asthma_shifted = ColumnDataSource(data=dict(TimePeriod=asthma_yearly['TimePeriod'], Percent_norm=asthma_yearly['Percent_norm']))

# 10️⃣ Crear figura
p = figure(title='Normalized Asthma vs PM2.5 (Cigarettes Equivalent + Labels)',
           x_axis_label='Year',
           y_axis_label='Normalized Asthma (0-1) / PM2.5 as Cigarettes/day',
           width=900, height=500,
           tools='pan,wheel_zoom,box_zoom,reset,save')

# 11️⃣ Barras PM2.5
bars_pm25 = p.vbar(x='TimePeriod', top='Cigarettes_per_day', source=source_pm25,
                   width=0.8, color='#9467bd', legend_label='PM2.5 (Cigarettes/day)', alpha=0.6)

# 12️⃣ Etiquetas encima barras con fondo semitransparente
labels = LabelSet(x='TimePeriod', y='Cigarettes_per_day', text='Cigarettes_per_year',
                  level='glyph', x_offset=-13, y_offset=3, source=source_pm25,
                  text_font_size="10pt", text_color="#000000",
                  background_fill_color='white', background_fill_alpha=0.6,
                  visible=True)
p.add_layout(labels)

# 13️⃣ Asma (Línea y círculos)
line1 = p.line('TimePeriod', 'Percent_norm', source=source_asthma_shifted, color='#ff7f0e', legend_label='Asthma (Normalized)', line_width=3)
circle1 = p.circle('TimePeriod', 'Percent_norm', source=source_asthma_shifted, color='#ff7f0e', size=8)

# 14️⃣ Hover tools
hover_asthma = HoverTool(tooltips=[('Year', '@TimePeriod'), ('Asthma Value (normalized)', '@Percent_norm{0.00}')], renderers=[line1, circle1])
hover_pm25 = HoverTool(tooltips=[('Year', '@TimePeriod'), ('PM2.5 (µg/m³)', '@Annual_mean{0.0}'), ('Cigarettes/day', '@Cigarettes_per_day{0.00}')], renderers=[bars_pm25])
p.add_tools(hover_asthma, hover_pm25)

# 15️⃣ Interactividad leyenda
p.legend.click_policy = "hide"

# 16️⃣ Toggle etiquetas
label_toggle = CheckboxButtonGroup(labels=["Show Cigarettes/year Labels"], active=[0])
callback_label = CustomJS(args=dict(labels=labels, button=label_toggle), code="""
    labels.visible = button.active.includes(0);
""")
label_toggle.js_on_change('active', callback_label)

# 17️⃣ Slider asma
shift_slider = Slider(start=-10, end=10, value=0, step=1, title="Shift Asthma Curve (Years)")
callback = CustomJS(args=dict(source=source_asthma_shifted, slider=shift_slider,
                               original_x=asthma_yearly['TimePeriod'].tolist(),
                               original_y=asthma_yearly['Percent_norm'].tolist()), code=""" 
    const data = source.data;
    const shift = slider.value;
    const x = original_x;
    const y = original_y;
    const new_x = [];
    for (let i = 0; i < x.length; i++) {
        new_x.push(x[i] + shift);
    }
    data['TimePeriod'] = new_x;
    source.change.emit();
""")
shift_slider.js_on_change('value', callback)

# 💾 Guardar como HTML
output_file("asthma_pm25_cigarettes_labels_final.html", title="Asthma vs PM2.5 (Cigarettes Equivalent + Labels)")

# 📊 Mostrar gráfico con slider y toggle
save(column(p, shift_slider, label_toggle))


'c:\\Users\\Eva María\\OneDrive - Danmarks Tekniske Universitet\\master\\1_year\\spring\\socialData\\finalprojectRepo\\ProjectB_Group55.github.io\\code and data\\asthma_pm25_cigarettes_labels_final.html'